In [39]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from sklearn.preprocessing import normalize


In [40]:
PERSON_FOLDER = "/home/sandeshprasai/Projects/Final_Semester_Project/AI_Attendance_System/ai-ml-model/DataSets/SandeshPrasai"
PERSON_NAME = "Sandesh Prasai"   


In [41]:
import tensorflow as tf

def l2_norm(x, axis=1):
    return tf.math.l2_normalize(x, axis=axis)

class ArcFace(tf.keras.layers.Layer):
    def __init__(self, num_classes, margin=0.5, scale=64, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.margin = margin
        self.scale = scale

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": self.num_classes,
            "margin": self.margin,
            "scale": self.scale,
        })
        return config


In [42]:
def load_preprocessed_face(img_path):
    img = cv2.imread(img_path)

    if img is None:
        raise ValueError("Image not readable")

    # BGR → RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # FORCE resize (CRITICAL FIX)
    img = cv2.resize(img, (112, 112), interpolation=cv2.INTER_AREA)

    # Normalize (matches training)
    img = img.astype(np.float32) / 255.0

    return img


In [43]:
def extract_person_embeddings(folder_path, embedding_model):
    embeddings = []

    image_files = [
        f for f in os.listdir(folder_path)
        if f.lower().endswith(('.jpg', '.png', '.jpeg'))
    ]

    print(f"Found {len(image_files)} images")

    for img_name in tqdm(image_files, desc="Extracting embeddings"):
        img_path = os.path.join(folder_path, img_name)

        try:
            img = load_preprocessed_face(img_path)
            img = np.expand_dims(img, axis=0)  # (1,112,112,3)

            emb = embedding_model.predict(img, verbose=0)
            embeddings.append(emb.flatten())

        except Exception as e:
            print(f"[SKIPPED] {img_name}: {e}")

    return np.array(embeddings)


In [44]:
embedding_model = tf.keras.models.load_model(
    r"/home/sandeshprasai/Projects/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models/RestNet50/final_year_project_face_recognition/embedding_model.keras",
    custom_objects={
        "l2_norm": l2_norm,
        "ArcFace": ArcFace
    },
    compile=False   # IMPORTANT
)


In [45]:
person_embeddings = extract_person_embeddings(
    PERSON_FOLDER,
    embedding_model
)

print("Raw embeddings shape:", person_embeddings.shape)


Found 9 images


Extracting embeddings: 100%|██████████| 9/9 [00:02<00:00,  3.10it/s]

Raw embeddings shape: (9, 512)


In [46]:
person_embeddings = normalize(person_embeddings, norm="l2")


In [47]:
person_embedding_mean = np.mean(person_embeddings, axis=0)

print("Final embedding shape:", person_embedding_mean.shape)


Final embedding shape: (512,)


In [48]:
np.save(f"{PERSON_NAME}_embedding.npy", person_embedding_mean)


## Match Embeddings 

In [49]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [50]:
REFERENCE_EMB_PATH = "/home/sandeshprasai/Projects/Final_Semester_Project/AI_Attendance_System/ai-ml-model/notebook/Verson_3_RestNet+ArcFace/Sandesh Prasai_embedding.npy"
reference_embedding = np.load(REFERENCE_EMB_PATH)

print("Reference embedding shape:", reference_embedding.shape)


Reference embedding shape: (512,)


In [51]:
from mtcnn import MTCNN

detector = MTCNN(device="cpu")


In [52]:
REFERENCE_FACIAL_POINTS = np.array([
    [38.2946, 51.6963],
    [73.5318, 51.5014],
    [56.0252, 71.7366],
    [41.5493, 92.3655],
    [70.7299, 92.2041]
], dtype=np.float32)


In [53]:
def align_face_to_template(img, landmarks):
    src_pts = np.array([
        landmarks['left_eye'],
        landmarks['right_eye'],
        landmarks['nose'],
        landmarks['mouth_left'],
        landmarks['mouth_right']
    ], dtype=np.float32)

    tform, _ = cv2.estimateAffinePartial2D(src_pts, REFERENCE_FACIAL_POINTS)
    if tform is None:
        return None

    return cv2.warpAffine(img, tform, (112, 112), borderValue=0)


In [54]:
def solve_lighting(img):
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    yuv[:, :, 0] = clahe.apply(yuv[:, :, 0])
    return cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)


In [55]:
def prepare_face_for_embedding(face_bgr):
    face_rgb = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
    face_rgb = face_rgb.astype(np.float32) / 255.0
    face_rgb = np.expand_dims(face_rgb, axis=0)  # (1,112,112,3)
    return face_rgb


In [56]:
cap = cv2.VideoCapture("http://192.168.1.75:4747/video")

THRESHOLD = 0.65  # start value (we can tune later)

print("Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(rgb)

    if results:
        best_face = max(results, key=lambda x: x['confidence'])

        aligned = align_face_to_template(frame, best_face['keypoints'])

        if aligned is not None:
            aligned = solve_lighting(aligned)

            face_input = prepare_face_for_embedding(aligned)

            emb = embedding_model.predict(face_input, verbose=0)
            emb = emb.flatten()
            emb = emb / np.linalg.norm(emb)  # L2 normalize

            similarity = cosine_similarity(
                emb.reshape(1, -1),
                reference_embedding.reshape(1, -1)
            )[0][0]

            label = "MATCH" if similarity >= THRESHOLD else "NO MATCH"

            x, y, w, h = best_face['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

            cv2.putText(
                frame,
                f"{label} | sim={similarity:.3f}",
                (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0,255,0),
                2
            )

    cv2.imshow("Face Verification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit


2026-02-04 12:20:35.180585: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
2026-02-04 12:20:37.496307: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
2026-02-04 12:20:38.419962: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
2026-02-04 12:20:40.578919: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
2026-02-04 12:20:43.367126: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]
2026-02-04 12:20:50.203399: I tensorflow/core/framework

In [57]:
cap.release()
cv2.destroyAllWindows()